In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [20]:
train=pd.read_csv("/content/drive/MyDrive/Minor Final/Dataset/Final_Dataset/Ginger.csv")
train.head()

,Commodity,Date,Average
0,Ginger,6/16/2013,145.0
1,Ginger,6/17/2013,145.0
2,Ginger,6/18/2013,145.0
3,Ginger,6/19/2013,145.0
4,Ginger,6/20/2013,145.0


In [21]:
train['Date']  = train['Date'].apply(pd.to_datetime)

In [22]:
train.isna().sum()

Commodity    0
Date         0
Average      0
dtype: int64

In [23]:
px.line(
    data_frame=train,
    x="Date",
    y="Average"
)

In [24]:
# resampled_df = train.resample("7D", on ='Date' , ).sum()
# px.line(
#     data_frame=resampled_df,
# #     x="Date",
#     y="Average"
# )

In [25]:
from sklearn.model_selection import train_test_split

y_train=train['Average']
# Splitting 'y_train' into training and testing sets without shuffling
y_train, y_test = train_test_split(y_train, test_size=0.2, shuffle=False)

In [26]:
y_train.head()

0    145.0
1    145.0
2    145.0
3    145.0
4    145.0
Name: Average, dtype: float64

In [27]:
# Training size:
print("Train set has {} observations.".format(len(y_train)))
# Test size:
print("Test set has {} observations.".format(len(y_test)))

Train set has 2889 observations.
Test set has 723 observations.


In [28]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=scaler.fit_transform(y_train.values.reshape(-1,1))
y_test=scaler.transform(y_test.values.reshape(-1,1))

In [29]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(0 , len(obs) - seq_size):
        window = obs[i:i + seq_size]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Using y_test only
x_train, y_train = to_sequences(SEQUENCE_SIZE, y_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, y_test)

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [30]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [31]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cuda


In [32]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model )
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers,enable_nested_tensor=True)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel(
    input_dim = 1,
    d_model = 32,
    nhead = 16,
    num_layers = 2,
    dropout= 0.05,
).to(device)

In [33]:
model

TransformerModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=32, out_features=1, bias=True)
)

In [34]:
from torch.optim.lr_scheduler import ConstantLR
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)
scheduler = ConstantLR(optimizer)
epochs = 100
early_stop_count = 0
min_val_loss = float('inf')
train_losses = []
validation_losses = []
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step()
    validation_losses.append(val_loss)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    # if early_stop_count >= 5:
    #     print("Early stopping!")
    #     break
    print(f"Epoch {epoch + 1}/{epochs},Train Loss {loss.item()} Validation Loss: {val_loss:.4f}")

Epoch 1/100,Train Loss 0.003255548421293497 Validation Loss: 0.2320
Epoch 2/100,Train Loss 0.002797216409817338 Validation Loss: 0.1599
Epoch 3/100,Train Loss 0.06217922270298004 Validation Loss: 0.1791
Epoch 4/100,Train Loss 0.00491801043972373 Validation Loss: 0.2292
Epoch 5/100,Train Loss 0.012677368707954884 Validation Loss: 0.2233
Epoch 6/100,Train Loss 0.004464366473257542 Validation Loss: 0.2050
Epoch 7/100,Train Loss 0.013260177336633205 Validation Loss: 0.2295
Epoch 8/100,Train Loss 0.021319128572940826 Validation Loss: 0.2546
Epoch 9/100,Train Loss 0.004118643701076508 Validation Loss: 0.2423
Epoch 10/100,Train Loss 0.005438740830868483 Validation Loss: 0.2158
Epoch 11/100,Train Loss 0.004496806301176548 Validation Loss: 0.2357
Epoch 12/100,Train Loss 0.004174362868070602 Validation Loss: 0.2391
Epoch 13/100,Train Loss 0.0033134028781205416 Validation Loss: 0.2672
Epoch 14/100,Train Loss 0.011677280068397522 Validation Loss: 0.2440
Epoch 15/100,Train Loss 0.005587855353951454

In [35]:
torch.save(model,"/content/drive/MyDrive/Minor Final/Transformer_Final model/Ginger.pth")

In [ ]:
model = torch.load("/content/drive/MyDrive/Minor Final/Transformer_Final model/Ginger.pth")

In [ ]:
criterion = nn.MSELoss()
model.eval()
predictions = []
valid_loss = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.tolist())
        validation_loss = criterion(outputs, y_batch)
        valid_loss.append(validation_loss.item())

# Convert predictions to a 2D numpy array
predictions = np.array(predictions).reshape(-1, 1)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calculate R^2 score
r2 = r2_score(y_test, predictions)

# Print scores
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

RMSE: 0.4786
MAE: 0.2496
MSE: 0.2291
R^2 Score: 0.9270
